In [1]:
# import libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import MapboxTiles
import geopandas as gpd
import random

In [2]:
# set user dir (WIP)
# user_dir = "/Users/ls/Dropbox (MIT)/02 Work/LCAU:SERC/Million Neighborhoods:DP/Data"

In [2]:
# load parcel and building data
# df_parcels = gpd.read_file("/Users/ls/Dropbox (MIT)/02 Work/LCAU:SERC/Million Neighborhoods:DP/Data/Parcels/NYC_2021_Tax_Parcels_SHP_2203/NewYork_2021_Tax_Parcels_SHP_2203.shp")
# df_buildings = gpd.read_file("/Users/ls/Dropbox (MIT)/02 Work/LCAU:SERC/Million Neighborhoods:DP/Data/Building Footprints/NYC/geo_export_a80ea1a2-e8e0-4ffd-862c-1199433ac303.shp")

df_parcels = gpd.read_file("data/NewYork_2021_Tax_Parcels_SHP_2203.shp")
df_buildings = gpd.read_file("data/geo_export_a80ea1a2-e8e0-4ffd-862c-1199433ac303.shp")

In [44]:
# set random RGB color for the parcels
def random_hex_color(seed=False):
  if seed:
    random.seed(seed)
    r = random.randint(0, 255)
    random.seed(seed+1000)
    g = random.randint(0, 255)
    random.seed(seed+2000)
    b = random.randint(0, 255)
  else:
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
  return "#{:02x}{:02x}{:02x}".format(r, g, b)
random_hex_color()
df_parcels['color'] = [ random_hex_color() for i in range(len(df_parcels)) ]

In [4]:
def join_parcels_buildings(parcels, buildings):
    """Join parcels and buildings dataframes."""
    parcels_buildings = buildings.sjoin(parcels, how="left", predicate="within")
    return parcels_buildings

In [6]:
# df_parcels.crs = {'init': 'epsg:26918'}
# df_buildings.crs = {'init': 'epsg:4326'}
# WGS84dd = {'init': 'epsg:4326'}

In [5]:
# set CRS to web mercator
df_parcels = df_parcels.to_crs(epsg=3857)
df_buildings = df_buildings.to_crs(epsg=3857)

In [6]:
df_parcels_buildings = join_parcels_buildings(df_parcels, df_buildings)
df_parcels_buildings['color'] = [ random_hex_color() for i in range(len(df_parcels_buildings)) ]

In [46]:
def add_parcels(ax, df_parcels, crs_epsg, random_color = False):
    # loop through the parcels and plot them
    for row in df_parcels.itertuples():
        geometry = row.geometry
        if random_color == True: color = random_hex_color(int(row.bin))
        else: color = row.color
        ax.add_geometries(geometry, crs = crs_epsg, facecolor=color) # for Lat/Lon data.

In [43]:
# load the background map and plot the geometries
# def map_maker(df_parcels, df_buildings, scale=10):
def map_maker(df_parcels, df_buildings, bounds, index, same_color=False, scale=10, feature_type='both', random_color=False):
    access_token = "pk.eyJ1IjoiZGVtaXJjYW50YXMiLCJhIjoiY2xrazNudjIzMDA5NjNvcG1lOHZuanVjMSJ9._t6VB8FA4_VAyJOJ9b92jQ"
    tiler = MapboxTiles(access_token, 'satellite-v9')
    crs_epsg = ccrs.epsg('3857')

    mercator = tiler.crs

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection=mercator)

    # change figure size of the subplot
    my_dpi=96
    fig.set_size_inches(7, 7)
    # fig.figsize = (512/my_dpi, 512/my_dpi), dpi=my_dpi

    # calculate the centroid and max distance of the bounds
    dist1 = bounds[2]-bounds[0]
    dist2 = bounds[3]-bounds[1]
    max_dist = max(dist1, dist2)/2

    # calculate the centroid of the bounds
    centroid_x = (bounds[2]+bounds[0])/2
    centroid_y = (bounds[3]+bounds[1])/2

    # bounds = df_parcels.total_bounds with offset to create same aspect ratio
    ax.set_extent([centroid_x-max_dist, centroid_x+max_dist, centroid_y-max_dist, centroid_y+max_dist], crs=ccrs.epsg('3857'))

    # if feature_type == 'parcels': add_parcels(ax, df_parcels, crs_epsg)
    if feature_type == 'both': 
        add_parcels(ax, df_parcels, crs_epsg)
        # ax.add_geometries(df_buildings.geometry, crs = crs_epsg, facecolor='white', edgecolor='black', linewidth=2.5, alpha=1)
    elif feature_type == 'both' and random_color == True:
        add_parcels(ax, df_parcels, crs_epsg, random_color=True)
    elif feature_type == 'buildings' and same_color == True: 
        add_parcels(ax, df_buildings, crs_epsg)
    elif  feature_type == 'buildings' and same_color == False: 
        ax.add_geometries(df_buildings.geometry, crs = crs_epsg, facecolor='black', edgecolor='white', linewidth=1.5, alpha=1)

    # add the Mapbox tiles to the axis at zoom level 10 (Mapbox has 23 zoom levels)
    ax.add_image(tiler, scale)

    # set the path to the folder where the images will be saved
    output_folder = 'img/'

    # save the figure
    plt.savefig(output_folder + f'parcel_{index}.jpg', bbox_inches='tight', pad_inches = 0, dpi = my_dpi)
    # plt.savefig(output_folder + f'building_{index}.jpg', bbox_inches='tight', pad_inches = 0, dpi = my_dpi)

    # close the figure
    plt.close(fig)

    # ax.coastlines('10m')
    plt.show()

In [11]:
# subset the data frames based on a buffer
def subset(df, df_buildings, index, distance = 75):
    selected_feature = df.loc[index]
    geometry_buffer = selected_feature.geometry.buffer(distance)
    geometry_bounds = selected_feature.geometry.buffer(distance-70)

    return df[df.within(geometry_buffer)], df_buildings[df_buildings.within(geometry_buffer)], geometry_bounds.bounds

#### Alternative subset function from [Colab Notebook](https://drive.google.com/file/d/1j1ao_z13UqCxdtX8_GjYH9MC-7PhNsth/view?usp=sharing)

In [29]:
# alternative subset function for faster rendering
def subset_alt(df, df_buildings, index, distance = 75):
    selected_feature = df.loc[index, 'geometry']
    geometry_bounds = selected_feature.total_bounds
    filtered_df = df.cx[geometry_bounds[0] - distance: geometry_bounds[2] + distance, geometry_bounds[1] - distance: geometry_bounds[3] + distance]
    filtered_df_buildings = df_buildings.cx[geometry_bounds[0] - distance: geometry_bounds[2] + distance, geometry_bounds[1] - distance: geometry_bounds[3] + distance]

    return filtered_df, filtered_df_buildings, geometry_bounds

In [19]:
# Index will correspond to each of the parcel indices in the data frame. 
# distance is in meters
# subset_features = subset(df_parcels, df_buildings, 1999, 200)
# map_maker(subset_features[0], subset_features[1], 18)

In [40]:
# create a folder to store the images
# os.mkdir('img')

# index will correspond to each of the parcel indices in the data frame. 
for i in range (180, 181):
    # subset_features = subset(df_parcels, df_parcels_buildings, i, 200)
    subset_features = subset(df_parcels, df_parcels_buildings, i, 200)
    map_maker(subset_features[0], subset_features[1], subset_features[2], i, True, 18,'buildings')


In [14]:
df_buildings

,name,bin,cnstrct_yr,date_lstmo,time_lstmo,lststatype,doitt_id,heightroof,feat_code,groundelev,shape_area,shape_len,base_bbl,mpluto_bbl,geomsource,globalid,geometry
0,NaN,3170958.0,1925.0,2017-08-22,00:00:00.000,Constructed,96807.0,29.749853,2100.0,40.0,0.0,0.0,3065220021,3065220021,Photogramm,{31298F86-3088-4F53-B3DB-71A9EFA6FA1F},"POLYGON ((-8233929.336 4957331.808, -8233951.9..."
1,NaN,5028452.0,1965.0,2017-08-22,00:00:00.000,Constructed,326368.0,22.630000,2100.0,39.0,0.0,0.0,5012640036,5012640036,Photogramm,{F5F8CDA5-69E2-46F8-8F69-BA95C025B520},"POLYGON ((-8256333.087 4959292.068, -8256333.3..."
2,NaN,5078368.0,1970.0,2017-08-22,00:00:00.000,Constructed,746627.0,35.760000,2100.0,51.0,0.0,0.0,5060190091,5060190091,Photogramm,{9F644794-F72C-4582-9E5E-B337E2B97068},"POLYGON ((-8259361.657 4947086.361, -8259371.9..."
3,NaN,3245111.0,1928.0,2017-08-22,00:00:00.000,Constructed,786626.0,37.500000,2100.0,6.0,0.0,0.0,3086910048,3086910048,Photogramm,{F916B22D-E25B-44AE-9FA9-2A51191B9CDF},"POLYGON ((-8233315.849 4950212.442, -8233317.6..."
4,NaN,4161096.0,1950.0,2017-08-22,00:00:00.000,Constructed,746409.0,18.015113,2100.0,93.0,0.0,0.0,4075020005,4075020005,Photogramm,{525F2C24-616B-4F29-98A3-8FEA5D4B1A7D},"POLYGON ((-8210281.723 4976405.244, -8210276.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084087,NaN,3384744.0,NaN,2023-06-23,00:00:00.000,Constructed,1299929.0,10.000000,5110.0,18.0,0.0,0.0,3066680054,3066680054,Other (Man,{1E2DAD17-9F3F-424C-9635-D81DA017AAC0},"POLYGON ((-8235922.107 4953644.890, -8235921.6..."
1084088,NaN,4624168.0,2020.0,2023-06-23,00:00:00.000,Constructed,1299934.0,10.000000,5110.0,11.0,0.0,0.0,4115550143,4115550143,Other (Man,{AA45367D-506C-46A2-A352-63430900B162},"POLYGON ((-8219443.587 4963643.527, -8219437.5..."
1084089,NaN,4619619.0,2022.0,2023-06-23,00:00:00.000,Constructed,1299931.0,60.000000,2100.0,78.0,0.0,0.0,4075290001,4075290001,Other (Man,{DB88DE9C-B7A5-4D01-8FAA-860D2DD2A06C},"POLYGON ((-8210614.224 4975745.792, -8210607.6..."
1084090,NaN,4623642.0,2019.0,2023-06-23,00:00:00.000,Constructed,1299932.0,10.000000,5110.0,112.0,0.0,0.0,4024980002,4024980002,Other (Man,{3033F535-DB93-4AC8-97E2-7ACED2D3D274},"POLYGON ((-8225957.517 4972489.433, -8225956.0..."


In [15]:
df_parcels_buildings

,name,bin,cnstrct_yr,date_lstmo,time_lstmo,lststatype,doitt_id,heightroof,feat_code,groundelev,...,SWISPKID,ROLL_YR,SPATIAL_YR,OWNER_TYPE,NYS_NAME,NAMESOURCE,DUP_GEO,CALC_ACRES,GFA,color
0,NaN,3170958.0,1925.0,2017-08-22,00:00:00.000,Constructed,96807.0,29.749853,2100.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#871585
1,NaN,5028452.0,1965.0,2017-08-22,00:00:00.000,Constructed,326368.0,22.630000,2100.0,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#6e53e9
2,NaN,5078368.0,1970.0,2017-08-22,00:00:00.000,Constructed,746627.0,35.760000,2100.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#8a7912
3,NaN,3245111.0,1928.0,2017-08-22,00:00:00.000,Constructed,786626.0,37.500000,2100.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#86f6b1
4,NaN,4161096.0,1950.0,2017-08-22,00:00:00.000,Constructed,746409.0,18.015113,2100.0,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#28e715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084087,NaN,3384744.0,NaN,2023-06-23,00:00:00.000,Constructed,1299929.0,10.000000,5110.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#b67140
1084088,NaN,4624168.0,2020.0,2023-06-23,00:00:00.000,Constructed,1299934.0,10.000000,5110.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#fa7037
1084089,NaN,4619619.0,2022.0,2023-06-23,00:00:00.000,Constructed,1299931.0,60.000000,2100.0,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#e363c8
1084090,NaN,4623642.0,2019.0,2023-06-23,00:00:00.000,Constructed,1299932.0,10.000000,5110.0,112.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#90627a
